# Additive Secret Sharing

**Module 12** | 12-mpc

*Random splits, XOR, reconstruction*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **additive secret sharing**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [Shamir Secret Sharing](12a-secret-sharing-shamir.ipynb).
- Concepts and notation introduced in the previous notebook.

## Additive Sharing over a Field

In [ ]:
# Split secret s into n shares: s = s_1 + s_2 + ... + s_n
# Pick s_1, ..., s_{n-1} randomly; s_n = s - sum(others)
# TODO: implement additive sharing

## XOR-Based Sharing

In [ ]:
# For binary data: s = s_1 XOR s_2 XOR ... XOR s_n
# TODO: implement XOR sharing and reconstruction

## Addition on Shared Values

In [ ]:
# To add shared values: each party adds their shares locally
# [a] + [b] = [a+b] without any communication!
# TODO: demonstrate local addition on shares

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **additive secret sharing**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [Yao's Garbled Circuits](12c-yaos-garbled-circuits.ipynb)